# Thinking Conditionally: Tradelanes and Modes
DA Probability & Statistics Learning Series • Lesson 2

<img src="https://imgs.xkcd.com/comics/conditionals.png" align="center"/>

Welcome, again!

*See **#da_prob_stat** for discussion during and after this tutorial.*


## Goals

**You**: Develop an intuition around conditional vs. unconditional probabilities
<br>
**You**: Invent some math to answer questions. Sorta feel badass about it.
<br>
**You**: Understand independence of events.
<br>
**You**: Get feet wet regarding the Law of Total Probability (LOTP).


---


**Me**: Do a better job of managing time!
<br>
**Me**: Take more time to explain code.
<br>
**Me**: Be explicit about what's to be done now vs. later for homework.
<br>
**Me**: Cold call more! 😈


Let's get started by bringing in the stuff we're going to need for this lesson.

In [ ]:
# Import dependencies
import sys
sys.path.insert(0, '../custom')
from db_utils import get_connection
import pandas as pd

# Object typing
from typing import TypeVar
PandasSeries = TypeVar('pd.core.series.Series')
PandasDataFrame = TypeVar('pd.core.frame.DataFrame')

# Data viz
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

In [ ]:
# Get the database connection and cursor objects
conn, cur = get_connection()

# Use a context manager to open and close connection and files
with conn:
    
    # Open the query.sql file
    with open('query.sql', 'r') as q:

        # Save contents of query.sql as string
        query_str = q.read()
    
    # Use the read_sql method to get the data from Snowflake into a 
    # Pandas dataframe
    df = pd.read_sql(query_str, conn)
    
    # Make all the columns lowercase
    df.columns = map(str.lower, df.columns)

# Preview the data
df.sample(3)

## Motivating Question 🤔

> Are shipments' tradelanes and modes independent of one another?


## Quick Review ⏪

We've already seen how to use counting and the Naïve Definition of Probability to calculate the probability of events in a discrete setting.

To calculate the $P(A)$, the probability of some event $A$ occurring, we just need to count the number of **outcomes** in **event** $A$ and divide this by the total number of outcomes found in the **sample space**, $S$.

### Conditional Probability

Let's start with a pretty straightforward example taken from the diagnostic quiz awhile back.

**Question**:

> Imagine a client makes exactly two shipments. Assume that for this client, any given shipment is transported either via ocean (event O) or by air (event A) with equal probability (i.e., P(O) = P(A) = 0.5) and that the choice of transportation for one shipment is independent of that of another.
>
> **Which of the following scenarios is more probable?**


A. Both shipments are air if we know at least one shipment is air.
<br>
B. Both shipments are air if we know the second shipment is air.
<br>
C. Both scenarios are equally probable.
<br>

Remember "pebble world" from last time?

We know that each outcome ("pebble") is equally likely ("weighs the same"). Now all we have to do is consider how each event differs.

![Quiz Question Sample Space](quiz_question_sample_space.png)

- The <font color='orange'>orange box</font> is the **event** that _at least one shipment is air_.
- The <font color='red'>red box</font> is the **event** that _the second shipment is air_. 
- The <font color='teal'>teal oval</font> is the **event** that _both shipments are air_. (Note, that this event only consists of one **outcome**.)

If we formalize this using the notation we learned last time and the naïve definition of probability, we have:

$$
\begin{align}
P(\text{at least one shipment is air}) &= \frac{3}{4}\\
P(\text{the second shipment is air}) &= \frac{1}{2}
\end{align}
$$

When we **condition** on evidence or information or what we observe, we need to update our probabilities to incorporate what we've learned. 

Said another way, we need to **adjust our our sample space (i.e., the denominator of the probability fraction) to include only the events (or outcomes) that meet the condition**.

The way we notationally represent this is by using a "|" to separate what we want to know (an event or events) from what we are conditioning on (another event or events). For this problem, the answer choices expressed in this notation are:

A. $P(\text{both air shipments}\,|\,\text{at least one shipment is air})$<br>
B. $P(\text{both air shipments}\,|\,\text{the second shipment is air})$<br>

How do we take the information we have from what we know (unconditionally) and use it to get these conditional probabilities?

...by redefining what we consider "possible" outcomes!

Let's tackle $P(\text{both air shipments}\,|\,\text{at least one shipment is air})$. We are **conditioning** on the event that *at least one shipment is air*, so let's update what we consider possible outcomes under this condition:

![Quiz Question Choice A](quiz_question_choice_A.png)

Then is becomes immediately obvious that $P(\text{both air shipments}\,|\,\text{at least one shipment is air}) = \frac{1}{3}$.

Similarly, $P(\text{both air shipments}\,|\,\text{the second shipment is air}) = \frac{1}{2}$.

![Quiz Question Choice B](quiz_question_choice_B.png)

From here, we can see that knowing more information about which shipment was air increases our belief that both shipments are air:

- **Unconditionally**, we assigned a 25% probability that both shipments were air, $P(\text{both shipments are air}) = \frac{1}{4}$.<br>
- If we knew that *at least one shipment was air*, we could update our priors and **condition on this information**, leading us to increase our belief to 33% that both shipments were air: $P(\text{both shipments are air}\,|\,\text{at least one shipment is air}) = \frac{1}{3}$.<br>
- If we knew that *the second shipment in particular was air*, we can **update our priors even more**, which leads to a stronger belief of 50% that both shipments were air: $P(\text{both shipments are air}\,|\,\text{at least one shipment is air}) = \frac{1}{2}$.

A _Bayesian_ statistician would say that we've **updated our prior beliefs** (i.e., our *priors*).

## Warm-up 💻

We're going to try our hand at determining the independence of events using simple data visualizations and a little logic. If we're lucky we'll have stumbled upon our first test statistic.

**TODO**: Explore the data a little. Try to illustrate the relationship between tradelane and mode.

In [ ]:
# Isolate data to be used
tradelane_mode_df = df[['tradelane', 'mode']]

# Preview the data
display(tradelane_mode_df.head())

In [ ]:
## TODO: Explore the data; some suggested questions to answer
#        are listed below.

# How big is the data?


# Check that the data contains no nulls


# How many unique values are in each columns?


# What are the unique values in each column?


**Question**: Answer the following qualitatively based on what you see in the data (i.e., no need to perform statistical tests or calculations).

> Is a shipment's tradelane independent of its mode?

(Knowing each row in our dataframe is a shipment, what would you want to look for? How would you need to transform your data?)

Well, right now we have basically two lists with each row representing one data point. We should categorize the data based on the features/variables/attributes we have on hand! 

Let's _cross-tabulate_ this table so we have rows representing `tradelanes` and columns representing `mode`. The values in each cell will be the counts of shipments.

Notice, by transforming our data this way, we've created a sample space consisting of disjoint events! Each cell is an event: its the _intersection_ of a particular `tradelane` and a particular `mode`.

**TODO**: Transform our data into something workable by cross-tabulating our data with `tradelane`s as rows and `mode`s as columns.

In [ ]:
## TODO: Cross-tabulate shipments
# (Read docs for Pandas' crosstab function)

# Use this variable to store cross-tabulation WITHOUT
# marginal distributions (see 'margins' parameter)
tradelane_mode_xt = 

# Use this variable to store cross-tabulation WITH
# marginal distributions (see 'margins' parameter)
tradelane_mode_xt_marg = 

display(tradelane_mode_xt)
display(tradelane_mode_xt_marg)

Below, I've provided a helper function that will plot a heatmap of the _joint distribution_.

In [ ]:
# Helper functions
def heatmap(data: PandasDataFrame, 
            A: str,
            B: str) -> None:
    '''
    Plots heatmap of bivariate frequency data, i.e., P(A|B) when supplied
    data, A (as a string) and B (as a string)
    '''
    fig, ax = plt.subplots()
    sns.heatmap(data, ax=ax, annot=True, fmt='.0%', 
                cmap='cividis', vmin=0, vmax=1)
    ax.set_title("$P({0}\,|\,{1})$".format(A, B), 
                 fontsize='x-large')
    cbar = ax.collections[0].colorbar
    cbar.ax.yaxis.set_major_formatter(PercentFormatter(1, 0))
    plt.show()

**TODO**: Let's condition on `mode` and see what the distribution looks like.

In [ ]:
## TODO: Convert to relative frequencies, conditioning on mode
data = 
heatmap(data, 'tradelane', 'mode')

# Compare to marginal frequency of tradelane
mf_tradelane = pd.DataFrame(tradelane_mode_xt_marg/tradelane_mode_xt.sum().sum())
mf_tradelane.loc[mf_tradelane.index[:-1], 
                 ['All']].style.format("{:.0%}").background_gradient(cmap='cividis', 
                                                                 axis=1, 
                                                                 vmin=0, 
                                                                 vmax=1)

**TODO**: Compare the _marginal distribution_ (on the bottom) versus each column in the heatmap above. 

**Question**: 

> What does this tell you about independence?

In [ ]:
# Convert to relative frequencies, conditioning on tradelane
data = tradelane_mode_xt.apply(lambda s: s/tradelane_mode_xt.sum(axis=1))
heatmap(data, 'mode', 'tradelane')

# Compare to marginal frequency of mode
mf_mode = pd.DataFrame(tradelane_mode_xt_marg.T/tradelane_mode_xt.sum().sum())
mf_mode.loc[mf_mode.index[:-1],
            ['All']].T.style.format("{:.0%}").background_gradient(cmap='cividis', 
                                                                  axis=1, 
                                                                  vmin=0, 
                                                                  vmax=1)

But you are all data analysts. You're supposed to measure things and tell us what they mean.

**Question**: 

> How would I go about measuring the extent to which these variables are independent?

**Meta Question**: 

> How does ~Abhi~ someone answer a question they may not be prepared for?

**Meta Answer**: 

> Make something up and talk about it until it starts becoming believable!

That being said, back to the original question:

> How would I go about measuring the extent to which these variables are independent?

In [ ]:
# These are the observed frequencies
print('Observed Frequencies:')
display(tradelane_mode_xt_marg.style.format("{:,.0f}"))

**Question**: 

> _If_ these two variables were independent, what values would I expect in each cell?

In [ ]:
# Useful function to get expected counts
from scipy.stats import chi2_contingency

# Get expected counts and make into dataframe matching our
# previous cross-tab
chi2_stat, p_value, dof, exp_counts = chi2_contingency(tradelane_mode_xt)
exp_tradelane_mode_xt = pd.DataFrame(exp_counts)
exp_tradelane_mode_xt.index = tradelane_mode_xt.index
exp_tradelane_mode_xt.columns = tradelane_mode_xt.columns

# Add 'All' total row & column
exp_tradelane_mode_xt['All'] = exp_tradelane_mode_xt.sum(axis=1)
exp_tradelane_mode_xt = exp_tradelane_mode_xt.append(
    exp_tradelane_mode_xt.sum(axis=0).rename('All'))

# These are the expected frequencies (if tradelane and mode were independent)
print('Expected Frequencies:')
exp_tradelane_mode_xt.style.format("{:,.0f}")


**TODO**: Now, look at the differences between what we _observed_ and what we _expected_ to see.

In [ ]:
# TODO: Subtract the expected from the observed frequencies
tradelane_mode_xt_diff = 
tradelane_mode_xt_diff.style.format("{:.0f}")

**Question**: 

> The errors are positive and negative. How can we generalize this _algorithm_ so the metric it produces is consistent?

**Question**: 

> But how do we deal with the absolute magnitude of these numbers? 

(We wouldn't want our metric to be bigger just because we have more counts of stuff.)

Great! So now we have the difference between observed and expected -- our _error_. 

But there are lots of numbers here, one for each cell, in fact.

**Question**: 

> How can we summarize this in one number?

Let's call our newly concocted metric something: how about the "$\chi^{2}$ statistic"?

**TODO**: Implement this statistic. 

In [ ]:
# TODO: Implement our metric so it matches the out-of-box calculation
our_chi2 =

print(f"Our Chi-Square statistic: {our_chi2:>20.2f}")
print(f"SciPy's Chi-Square statistic: {chi2_stat:>16.2f}")


![Nice](https://media.giphy.com/media/8xgqLTTgWqHWU/giphy.gif)

Nice. We just invented some math that's somewhat legit.

Look familiar? This was in the prompt of the last question on the diagnostic quiz re: interpreting $p$-values)!

Later, once we have the fundamentals down, we'll get to statistical hypothesis testing. But let's look at the $p$-value for our test statistic here:

In [ ]:
display(dof)
display(p_value)

As we will explore much later (we should work to get the fundamentals down first), **the $p$-value is a _conditional_ probability itself**!

**Question**: 

> Under what condition is the $p$-value?

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/P-value_in_statistical_significance_testing.svg/1024px-P-value_in_statistical_significance_testing.svg.png" width=500/>

## Combining Events 🌗 🌓 🌕

An **event** is just a subset of the **sample space**. So when we _union_ or _intersect_ events, we're just doing normal set operations.

### Intersection of Events $\cap$

$$
P(A \cap B) = P(B)P(A|B) = P(A)P(B|A)
$$

Why is this true?

![Conditional Probability in "Pebble World"](./pebble_world_conditioning.png)

### Union of Disjoint Events

When events are *disjoint*, taking their union is straightforward since there is no overlap in events. All we do is sum up the disjoint events (like we did with the shipments-tradelane problem last time). 

![Third Axiom of Probability](https://wikimedia.org/api/rest_v1/media/math/render/svg/47f22fe03df467b1d20785e5026bac39fabd9edc)

In fact, this is one of the [Axioms of Probability](https://en.wikipedia.org/wiki/Probability_axioms)!

...but things get trickier when events do overlap (read: intersect)!

### Union of Events $\cup$

**Question**

> So, what do we do when events overlap?

![2 overlapping events](https://www.probabilitycourse.com/images/chapter1/union_b.png)

<img src="https://www.conceptdraw.com/How-To-Guide/picture/3-circle-venn.png" width=700 />

**Answer**: 

> Principle of Inclusion-Exclusion
>
> ![PIE](https://wikimedia.org/api/rest_v1/media/math/render/svg/c14e130250afb7484a5162c7fc76fab425397279)

This isn't as crazy as it looks. Sit and think about what it's doing: try some simple examples, try to explain it to a roommate, or even write some Python if you have to!

**Homework**:

> Let's say SF Flexporters return to a post-COVID working arrangement wherein each of the 20 DAs are "buddied up" with one other DA. Effectively, there are two shifts: a Monday shift (10 DAs) and a Friday shift consisting of the other 10 DAs (i.e., the buddies of those in the first shift). 
>
> Assume that seating is random and independent (i.e., no coordination of seats between buddies or among those in a shift) -- people just show up and sit somewhere.
>
> **What's the probability that no one in the Friday shift is sitting in the same seat as their \[Monday shift\] buddy?**

(Remember, you have a computer. Try to simulate this if the analytical solution isn't coming to you. **Work together!**)

## Independence ⫫

Now we can introduce, more formally, what **independence** means among events. Later we'll extend this notion to random variables.

### Definition of Independence

<br>

$A$ and $B$ are **independent** of one another if and only if $P(A \cap B) = P(A)P(B)$. (Sometimes this is written as $P(A,B) = P(A)P(B)$.)

Said another way, $A$ and $B$ are **independent** of one another if and only if $P(A\,|\,B) = P(A)$, which is equivalent to $P(B\,|\,A) = P(B)$.

In plain-speak, this just says that having information about one event occurring (or not*) doesn't change the probability of the other event.

<div>
<img src="https://courses.cs.cornell.edu/cs2800/wiki/images/thumb/3/3b/Conditional-probability.svg/445px-Conditional-probability.svg.png" width="800"/>
</div>

<br>

As mentioned before, **conditioning just re-normalizes our set of possible outcomes given new information**. This provides the basis for translating between unconditional probabilities and conditional probabilities.


Going back to our diagnostic quiz question about ocean and air shipments, we can use this formalization to calculate the probabilities of each scenario directly:

_Scenario A_

$$
\begin{align}
P(\text{both shipments are air}\,|\,\text{at least one shipment is air}) &= \\
\frac{P(\text{both shipments are air} \cap \text{at least one shipment is air})}{P(\text{at least one shipment is air})} &= \frac{P(\text{both shipments are air})}{P(\text{at least one shipment is air})} \\
&= \frac{\frac{1}{4}}{\frac{3}{4}} \\
&= \frac{1}{3}
\end{align}
$$

_Scenario B_

$$
\begin{align}
P(\text{both shipments are air}\,|\,\text{the second shipment is air}) &=\\ 
\frac{P(\text{both shipments are air} \cap \text{the second shipment is air})}{P(\text{the second shipment is air})} &= \frac{P(\text{both shipments are air})}{P(\text{the second shipment is air})} \\
&= \frac{\frac{1}{4}}{\frac{1}{2}} \\
&= \frac{1}{2}
\end{align}
$$

So, are the events in the two scenarios here independent?

**Questions**:

> Is the event "both shipments are air" independent of "at least one shipment is air"?

> Is the event "both shipments are air" independent of "the second shipment is air"?


**Homework**:

> \* Prove that $P(A, B) = P(A)P(B) \implies P(A\,|\,B^c) = P(A)$. 

In plain-speak, I'm asking you to prove that if an event is indepedent of another event having occurring, it is also indepedent of the other event not occurring.

Intuitively, it should make sense that if two things are independent, whether one of them happens _or not_ shouldn't affect the other, vice versa.

(Hint: Use what you already know about complements.)


## The Law of Total Probability 📜

Now we need connect conditional and unconditional probabilities. We do this with **the Law of Total Probability** (LOTP). 

<br>

Once we understand this, we will have all the required tools to prove the all-important identity: **Bayes' Theorem**, which we'll leave for next time. 

<br>

You'll also have the tools to deal with conditioning on multiple events/pieces of information since the concepts translate generally.



**The Law of Total Probability** is an incredibly useful problem solving tool. Formally stated, it says:

$$
\text{If }A_i,...,A_n \text{ is a partition of the sample space }S \text{, then }P(B) = \sum_{i=1}^{n}{P(B|A_i)P(A_i)}.
$$

But this is likely better illustrated with a picture:

![Partition of B by A](./LOTP.png)

Okay, your turn to practice!

**Question**: 

> What's $P(\text{TPEB})$. 

Partition the data and use LOTP so you can calculate it. Check against the data directly.

In [ ]:
## TODO: Demonstrate LOTP on our data; start with tradelane_mode_xt

# This is the denominator to convert cardinality of sets to probabilities
# (per the Naive Definition of Probability)
S = tradelane_mode_xt.sum().sum()

# Show that p_TPEB_by_LTOP == p_TPEB
p_TPEB = tradelane_mode_xt.loc['TPEB',:].sum()/S

p_Air = 

p_not_Air = 1 - p_Air

p_TPEB_given_Air = 

p_TPEB_given_not_Air = 


p_TPEB_by_LOTP = 

# Check if our answer is right   
print(f"Our Answer: {p_TPEB_by_LOTP:.5%}")
print(f"Expected Answer: {p_TPEB:.5%}")

**Homework**: 

> How would I get $P(\text{Ocean} \cup \text{Truck - Domestic}\,|\,\text{TAWB}\cup\text{Latin NB})$? 

> What can you say when you compare this with the unconditional probability, $P(\text{Ocean} \cup \text{Truck - Domestic})$?

> Is this the same thing as $P(\text{TAWB}\cup\text{Latin NB}\,|\,\text{Ocean} \cup \text{Truck - Domestic})$? Why or why not?

Oh, yeah...**work together**!

In [ ]:
## TODO: P(Ocean or Truck - Domestic | TAWB or Latin NB) = ?
# Not that its necessarily useful, but notice the unions are of 
# disjoint events (a shipment cannot be TAWB and Latin NB)

# Let A = Ocean or Truck - Domestic
# P(A)
p_A = 

# Let B = TAWB or Latin NB
# P(B)
p_B = 

# P(A, B)
p_A_and_B = 

print(f"P(Ocean or Truck - Domestic | TAWB or Latin NB) = {p_A_and_B/p_B:.2%}")
print(f"P(Ocean or Truck - Domestic) = {p_A:.2%}")